In [29]:
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
import datetime
import sqlite3

In [27]:
DATABASE_LOCATION = "sqllite:///my_played_tracks.sqlite"
USER_ID = "rezart392"
TOKEN = "BQChCbIhgqi9HeAUMEFjlKLnaJVIzzbMV6PmPZbYHBfk_6OaspCvh0CaUCRrbx6elKAexom48XOhXMmg980pLHGfHRIO7krWtHR_d3naw2IjnFUw08brcljCndsjNX2FvVUhHoBOlA2TOrE"

In [41]:
if __name__ == "__main__":

    headers = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : "Bearer {token}".format(token=TOKEN)
    }

    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days=1)
    yesterday_unix_timestamp = int(yesterday.timestamp())

    r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time=yesterday_unix_timestamp), headers = headers)

    data = r.json()

    song_names = []
    artist_names = []
    played_at_list = []
    timestamps = []


    for song in data["items"]:
        song_names.append(song["track"]["name"])
        artist_names.append(song["track"]["album"]["artists"][0]["name"])
        played_at_list.append(song["played_at"][0:10])
        timestamps.append(song["played_at"][0:10])

    song_dict = {
        "song_name" : song_names,
        "artist_name" : artist_names,
        "played_at" : played_at_list,
        "timestamp" : timestamps
    }


    song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])
    
    print(song_df)



                             song_name               artist_name   played_at  \
0                             Budapest               George Ezra  2022-03-28   
1            Blame (feat. John Newman)             Calvin Harris  2022-03-28   
2      Waves - Robin Schulz Radio Edit                 Mr. Probz  2022-03-28   
3                                 Gomd                   Sickick  2022-03-28   
4                              Pompeii                  Bastille  2022-03-28   
5                                Faded               Alan Walker  2022-03-28   
6            Here Comes the Hotstepper           Various Artists  2022-03-18   
7                               Angels           Robbie Williams  2022-03-18   
8                             Bailando                 Paradisio  2022-03-18   
9                         It Wasn't Me                    Shaggy  2022-03-18   
10                      Cotton Eye Joe                    Rednex  2022-03-18   
11                                Torn  